In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field

In [3]:
load_dotenv()

llm = ChatOpenAI( model="Qwen/Qwen2.5-7B-Instruct:together", api_key=os.environ["HF_TOKEN"], base_url="https://router.huggingface.co/v1" )


In [4]:
class SentimentSchema(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description = 'Sentiment of the review')

In [ ]:
structured_model = llm.with_structured_output(SentimentSchema)


In [8]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal['positive', 'negative']
    diagnosis: dict
    response: str

In [9]:
def find_sentiment(state: ReviewState):
    prompt = f'For the following review find out the sentiment \n {state["review"]}'
    sentiment = structured_model.invoke(prompt)
    return {"sentiment": sentiment}

In [ ]:
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment',find_sentiment)

